### Настройки

Начинаем с импорта библиотек

In [225]:
import pandas as pd
import psycopg2

In [226]:
# хороший тон при фиксации изменений в github не публиковать код с открытыми данными для входа
# для запуска кода поместите данные, которые указаны на странице курса в эту ячейку
# при размещении в github удалите эту ячейку
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

Создаем соединение с заданными параметрами

In [227]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

Запрос

In [13]:
# код запроса представляет из себя строковую переменную 
# 5.2
query = f'''select a.name, count(distinct e.id) cnt_emp, count(distinct v.id) cnt_vac
            from
                  areas a
                  left join employers e on a.id = e.area
                  left join vacancies v on v.area_id = a.id
            where v.id is null
            group by a.id
            order by cnt_emp desc
         '''
# лучше использовать f-строки - так удобнее передавать параметры

### Выполнение запроса

In [14]:
# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма 
# 5.2
df = pd.read_sql_query(query, connection)
df

C:\Users\ilyad\AppData\Local\Temp\ipykernel_13568\2473184231.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,name,cnt_emp,cnt_vac
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0
...,...,...,...
588,Скидель,0,0
589,Белоозерск,0,0
590,Орловский,0,0
591,Павловская,0,0


In [8]:
# код запроса представляет из себя строковую переменную 
# 5.4
query_5_4 = f'''select count(*) cnt
from employers e
     left join employers_industries ei on e.id = ei.employer_id
where ei.industry_id is null
                '''
# лучше использовать f-строки - так удобнее передавать параметры

In [9]:
# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма 
# 5.4
df = pd.read_sql_query(query_5_4, connection)
df

C:\Users\ilyad\AppData\Local\Temp\ipykernel_8780\1651684398.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_4, connection)


,cnt
0,8419


In [38]:
# код запроса представляет из себя строковую переменную 
# 5.5
query_5_5 = f'''select count(ei.industry_id) количество_сфер, e.name name_работодателя
from employers e
left join employers_industries ei on e.id = ei.employer_id
group by e.id
having count(ei.industry_id)=4
order by name_работодателя
offset 2 limit 1
                '''
# лучше использовать f-строки - так удобнее передавать параметры

In [39]:
# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма 
# 5.5
df = pd.read_sql_query(query_5_5, connection)
df

C:\Users\ilyad\AppData\Local\Temp\ipykernel_8780\731898222.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_5, connection)


,количество_сфер,name_работодателя
0,4,2ГИС


In [82]:
# код запроса представляет из себя строковую переменную 
# 5.6
query_5_6 = f'''select count(ei.industry_id) количество_сфер
from industries i
left join employers_industries ei on i.id = ei.industry_id
group by i.id
having i.name='Разработка программного обеспечения'
                '''
# лучше использовать f-строки - так удобнее передавать параметры

In [83]:
# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма 
# 5.6
df = pd.read_sql_query(query_5_6, connection)
df

C:\Users\ilyad\AppData\Local\Temp\ipykernel_8780\3363201357.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_6, connection)


,количество_сфер
0,3553


In [138]:
# код запроса представляет из себя строковую переменную 
# 5.7
import requests
from bs4 import BeautifulSoup

url='https://ru.wikipedia.org/wiki/Города-миллионеры_России'
response=requests.get(url)
page=BeautifulSoup (response. text, 'html.parser')

cities_here=str(page.find( 'table', class_="standard"))
cities_here=cities_here.split('title="')[3:]

million_cities = []
#million_cities = tuple(million_cities) 
for i in cities_here:
    million_cities.append(i.split('">')[0])
# лучше использовать f-строки - так удобнее передавать параметры

In [184]:
# код запроса представляет из себя строковую переменную 
# 5.7
query_5_7 = f'''
WITH temp AS 
            (select a.name as name, count(v.id) as cnt
            from employers e
            join vacancies v on e.id = v.employer_id
            join areas a on a.id = v.area_id
            where e.name = 'Яндекс' and a.name in {tuple(million_cities)}
            GROUP BY a.id)
            SELECT temp.name,
                    temp.cnt
            FROM temp
            UNION ALL
            SELECT 'Total', 
                    sum(temp.cnt)  
            FROM temp
'''
# лучше использовать f-строки - так удобнее передавать параметры

In [185]:
# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма 
# 5.7
df = pd.read_sql_query(query_5_7, connection)
df

C:\Users\ilyad\AppData\Local\Temp\ipykernel_8780\4037940136.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_7, connection)


,name,cnt
0,Москва,54.0
1,Санкт-Петербург,42.0
2,Екатеринбург,39.0
3,Новосибирск,35.0
4,Волгоград,24.0
5,Воронеж,32.0
6,Краснодар,30.0
7,Красноярск,23.0
8,Нижний Новгород,36.0
9,Омск,21.0


In [186]:
# код запроса представляет из себя строковую переменную 
# 6.1
query_6_1 = f'''select count(*)
                from vacancies
                where name ilike '%data%' or name ilike '%данн%'
                '''
# лучше использовать f-строки - так удобнее передавать параметры

In [187]:
# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма 
# 6.1
df = pd.read_sql_query(query_6_1, connection)
df

C:\Users\ilyad\AppData\Local\Temp\ipykernel_8780\430608920.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_1, connection)


,count
0,1771


In [188]:
# код запроса представляет из себя строковую переменную 
# 6.2
query_6_2 = f'''select count(*)
                from vacancies
                where (lower(vacancies.name) LIKE '%data scientist%' 
                or lower(vacancies.name) LIKE '%data science%' 
                or lower(vacancies.name) LIKE '%исследователь данных%' 
                or vacancies.name LIKE '%ML%' and vacancies.name NOT LIKE '%HTML%' 
                or lower(vacancies.name) LIKE '%machine learning%' 
                or lower(vacancies.name) LIKE '%машинн%обучен%') 
                and(lower(vacancies.name) LIKE '%junior%'
                or lower(vacancies.experience)='нет опыта'
                or lower(vacancies.employment)='стажировка')
                '''
# лучше использовать f-строки - так удобнее передавать параметры

In [189]:
# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма 
# 6.2
df = pd.read_sql_query(query_6_2, connection)
df

C:\Users\ilyad\AppData\Local\Temp\ipykernel_8780\2825064127.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_2, connection)


,count
0,51


In [190]:
# код запроса представляет из себя строковую переменную 
# 6.3
query_6_3 = f'''select count(*)
                from vacancies
                where (lower(vacancies.name) LIKE '%data scientist%' 
                or lower(vacancies.name) LIKE '%data science%' 
                or lower(vacancies.name) LIKE '%исследователь данных%' 
                or lower(vacancies.name) LIKE '%ml%' and vacancies.name NOT LIKE '%HTML%' and vacancies.name not like '%QML%' 
                or lower(vacancies.name) LIKE '%machine learning%' 
                or lower(vacancies.name) LIKE '%машинн%обучен%') 
                and(lower(vacancies.key_skills) LIKE '%postgres%'
                or lower(vacancies.key_skills) LIKE '%sql%')
                '''
# лучше использовать f-строки - так удобнее передавать параметры

In [191]:
# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма 
# 6.3
df = pd.read_sql_query(query_6_3, connection)
df

C:\Users\ilyad\AppData\Local\Temp\ipykernel_8780\3709805219.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_3, connection)


,count
0,229


In [194]:
# код запроса представляет из себя строковую переменную 
# 6.4
query_6_4 = f'''select count(*)
                from vacancies
                where (lower(vacancies.name) LIKE '%data scientist%' 
                or lower(vacancies.name) LIKE '%data science%' 
                or lower(vacancies.name) LIKE '%исследователь данных%' 
                or lower(vacancies.name) LIKE '%ml%' and vacancies.name NOT LIKE '%HTML%' and vacancies.name not like '%QML%' 
                or lower(vacancies.name) LIKE '%machine learning%' 
                or lower(vacancies.name) LIKE '%машинн%обучен%') 
                and(lower(vacancies.key_skills) LIKE '%python%')
                '''
# лучше использовать f-строки - так удобнее передавать параметры

In [195]:
# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма 
# 6.4
df = pd.read_sql_query(query_6_4, connection)
df

C:\Users\ilyad\AppData\Local\Temp\ipykernel_8780\1639840439.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_4, connection)


,count
0,357


In [214]:
# код запроса представляет из себя строковую переменную 
# 6.5
query_6_5 = f'''SELECT 
    round(avg(length(key_skills) - length(replace(key_skills, CHR(9), ''))+1), 2)
FROM vacancies
WHERE
    (lower(name) like '%data scientist%'
    or lower(name) like '%data science%'
    or lower(name) like '%исследователь данных%'
    or (name like '%ML%' and name not ilike '%HTML%')
    or lower(name) like '%machine learning%'
    or lower(name) like '%машинн%обучен%')
    and upper(name) not like '%HTML%'
    and key_skills is not null
                '''
# лучше использовать f-строки - так удобнее передавать параметры

In [215]:
# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма 
# 6.5
df = pd.read_sql_query(query_6_5, connection)
df

C:\Users\ilyad\AppData\Local\Temp\ipykernel_8780\2940343815.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_5, connection)


,round
0,6.41


In [228]:
# код запроса представляет из себя строковую переменную 
# 6.6
query_6_6 = f'''SELECT experience, round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to))) as wage
FROM vacancies 
WHERE (name ilike '%data science%'
    or name ilike '%data scientist%'
    or name ilike '%machine learning%'
    or name ilike '%исследователь данных%'
    or name ilike '%машинн%обучен%'
    or (name like '%ML%' and name not like '%HTML%'))   
GROUP BY 1 
                '''
# лучше использовать f-строки - так удобнее передавать параметры

In [229]:
# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма 
# 6.6
df = pd.read_sql_query(query_6_6, connection)
df

C:\Users\ilyad\AppData\Local\Temp\ipykernel_8780\4110104214.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_6, connection)


,experience,wage
0,Более 6 лет,NaN
1,Нет опыта,74643.0
2,От 1 года до 3 лет,139675.0
3,От 3 до 6 лет,243115.0


### Финал

In [224]:
# не забываем закрыть соединение после окончания работы
connection.close()